In [1]:
from langgraph.prebuilt import create_react_agent

In [33]:
import os
import pandas as pd

In [3]:
anthropic_model="anthropic:claude-3-7-sonnet-latest"

In [4]:
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

In [5]:
agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",
    tools=[get_weather],
    prompt="You are a helpful assistant"
)

In [6]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='cdcf7bef-3e28-4a66-9158-def39c3ae39a'),
  AIMessage(content=[{'text': "I'd be happy to check the weather in San Francisco for you. Let me do that now.", 'type': 'text'}, {'id': 'toolu_01Vu3z2c35qJPKnNZWVwP6TH', 'input': {'city': 'San Francisco'}, 'name': 'get_weather', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01NU8T811eYQH7xPXPDaHFUG', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 386, 'output_tokens': 75, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--906261a2-1af5-486d-8dd9-c516845dfcb0-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 't

### Tool Agent

In [7]:
from langchain_community.tools import DuckDuckGoSearchRun
import yfinance as yf
from langchain.agents import AgentExecutor, initialize_agent, Tool, AgentType

#### tools

In [8]:
# Web search tool
search = DuckDuckGoSearchRun()
search_tool = Tool(
    name="WebSearch",
    func=search.run,
    description="Search the web and return relevant results"
)


In [9]:
# Yahoo Finance tool

def get_stock_price(ticker: str) -> str:
    stock = yf.Ticker(ticker)
    price = stock.info['regularMarketPrice']
    return f"{ticker} price is {price}"

yahoo_tool = Tool(
    name="YahooFinance",
    func=get_stock_price,
    description="Get the latest stock price for a given ticker symbol"
)

In [10]:
get_stock_price('ASML')

'ASML price is 946.94'

In [11]:
# OTHER APIs & TOOLS HERE IN THE FUTURE

# ]Python REPL tool
def python_eval(code: str) -> str:
    try:
        return str(eval(code))
    except Exception as e:
        return f"Error: {e}"

python_tool = Tool(
    name="PythonREPL",
    func=python_eval,
    description="Run Python code and return the result"
)

#### fred api (federal reserve data)

In [12]:
# api variable set in env FRED_API_KEY

In [51]:
from fredapi import Fred
from datetime import datetime
def get_gdp_data(start_date: str = "2008-09-15",
                 end_date: str = datetime.now().strftime("%Y-%m-%d")
                ):
    """
    Fetches USA GDP data from the FRED API.

    Args:
        start_date: The start date for the data in YYYY-MM-DD format.
        end_date: The end date for the data in YYYY-MM-DD format.

    Returns:
        A string representation of the GDP data.
    """

    fred_api = os.getenv("FRED_API_KEY")
    if not fred_api:
        return "FRED_API_KEY not found. Please se the FRED_API_KEY environment variable"
        
    fred = Fred(fred_api)
    try:
        
        gpd_series = fred.get_series('GDP', start_date, end_data)
        gdp_df = pd.DataFrame({'date': gpd_series.index,
                                'GDP': gpd_series}
                            ).reset_index(drop=True)
    
        return gdp_df.to_string()


    except Exception as e:
        return f"An error occurred: {e}"
    
    

In [52]:
gdp_tool = Tool(
    name="GetGDP",
    func=get_gdp_data,
    description="Retrieve USA GDP data for a given date range"
)

In [43]:
series = fred.get_series('GDP')

In [44]:
series.index

DatetimeIndex(['1946-01-01', '1946-04-01', '1946-07-01', '1946-10-01',
               '1947-01-01', '1947-04-01', '1947-07-01', '1947-10-01',
               '1948-01-01', '1948-04-01',
               ...
               '2023-01-01', '2023-04-01', '2023-07-01', '2023-10-01',
               '2024-01-01', '2024-04-01', '2024-07-01', '2024-10-01',
               '2025-01-01', '2025-04-01'],
              dtype='datetime64[ns]', length=318, freq=None)

In [46]:
pd.DataFrame({'date': series.index,
                'GDP': series}
            ).reset_index(drop=True)

,date,GDP
0,1946-01-01,NaN
1,1946-04-01,NaN
2,1946-07-01,NaN
3,1946-10-01,NaN
4,1947-01-01,243.164
...,...,...
313,2024-04-01,29147.044
314,2024-07-01,29511.664
315,2024-10-01,29825.182
316,2025-01-01,30042.113


In [36]:
df = pd.DataFrame(fred.get_series('GDP'))
df.index = pd.to_datetime(df.index).strftime("%Y-%m-%d")

In [41]:
df.reset_index(names='date')

,date,0
0,1946-01-01,NaN
1,1946-04-01,NaN
2,1946-07-01,NaN
3,1946-10-01,NaN
4,1947-01-01,243.164
...,...,...
313,2024-04-01,29147.044
314,2024-07-01,29511.664
315,2024-10-01,29825.182
316,2025-01-01,30042.113


In [28]:
fred.get_series.__doc__

"\n        Get data for a Fred series id. This fetches the latest known data, and is equivalent to get_series_latest_release()\n\n        Parameters\n        ----------\n        series_id : str\n            Fred series id such as 'CPIAUCSL'\n        observation_start : datetime or datetime-like str such as '7/1/2014', optional\n            earliest observation date\n        observation_end : datetime or datetime-like str such as '7/1/2014', optional\n            latest observation date\n        kwargs : additional parameters\n            Any additional parameters supported by FRED. You can see https://api.stlouisfed.org/docs/fred/series_observations.html for the full list\n\n        Returns\n        -------\n        data : Series\n            a Series where each index is the observation date and the value is the data for the Fred series\n        "

In [29]:
fred.get_series_latest_release.__doc__

"\n        Get data for a Fred series id. This fetches the latest known data, and is equivalent to get_series()\n\n        Parameters\n        ----------\n        series_id : str\n            Fred series id such as 'CPIAUCSL'\n\n        Returns\n        -------\n        info : Series\n            a Series where each index is the observation date and the value is the data for the Fred series\n        "

In [17]:
#dir(fred)

In [38]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

REACT_TOOL_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an expert analyst and researcher. Using ONLY the tools provided, follow these steps carefully:
1. Perform any pre-processing or data gathering needed.
2. Use the appropriate tool(s) to answer or analyze the user's request.
3. Chain tools or aggregate intermediate results if needed.
4. Summarize your findings in a concise and useful way.

Reminder: Do NOT make up information—use only direct tool outputs.
"""),
    MessagesPlaceholder("messages"),  # 
    MessagesPlaceholder("agent_scratchpad", optional=True),
])

In [39]:

# Tool execution agent


tools = [search_tool, yahoo_tool, python_tool, gdp_tool]  # Pass tools above to pecialist list here

# Create tool

tools_executor = create_react_agent(
    model=anthropic_model,
    tools=tools,
    prompt=REACT_TOOL_PROMPT
)

In [41]:
tools_executor.invoke({
    "messages": [
        {"role": "user", "content": "what is the price of ASML"}
    ]
})

{'messages': [HumanMessage(content='what is the price of ASML', additional_kwargs={}, response_metadata={}, id='d492f4f3-c960-46e2-93da-1632f56550c3'),
  AIMessage(content=[{'text': "I'll check the current stock price of ASML for you using the YahooFinance tool.", 'type': 'text'}, {'id': 'toolu_01YbfJVPrPsiMbFdKTovWdfW', 'input': {'__arg1': 'ASML'}, 'name': 'YahooFinance', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01PEnPUjxX6kpDWJXtcWzWm4', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 631, 'output_tokens': 82, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--827f7a97-425a-4d9f-8309-e7c723bbfd97-0', tool_calls=[{'name': 'YahooFinance', 'args': {'__arg1': 'ASML'}, 'id': 'toolu_01YbfJV